In [ ]:
import os
import utils
import random
import dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [ ]:
# configst
#tfe.enable_eager_execution()
sess=tf.Session()
%matplotlib inline

# load data
os.system("sh clean.sh")
X=dataset.load_dataset("./ShapeNet",25,num_of_examples=1)
Y=None;

# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

In [ ]:
# encoder network
cur_tensor=tf.convert_to_tensor(X[0],dtype=tf.float32)
print(cur_tensor.shape)
k_s = [3,3]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with tf.name_scope("encoding_block"):
        cur_tensor=tf.layers.conv2d(cur_tensor,padding='VALID',filters=conv_filter_count[i],kernel_size= k_s)
        cur_tensor=tf.layers.max_pooling2d(cur_tensor,2,2)
        cur_tensor=tf.nn.relu(cur_tensor)
        print(cur_tensor.shape)
    
# flatten tensor
cur_tensor=tf.contrib.layers.flatten(cur_tensor)
print(cur_tensor.shape)
cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024)
print(cur_tensor.shape)

In [ ]:
# RNN just 1 cell now
state=tf.random_normal([1,1024])
cell=tf.nn.rnn_cell.GRUCell(1024)
cell(cur_tensor,state)
print(cur_tensor.shape)

# recive input from neightboars via 3*3 conv
#hidden_state_1=cur_tensor,tf.random_uniform([3,3,3])
#print(cur_tensor.shape)

In [ ]:
# collect hidden state from grid
hidden_states=tf.random_uniform([25,32,32,32])
cur_tensor=hidden_states
print(cur_tensor.shape)

k_s = [3,3]
for i in range(6): 
    with tf.name_scope("encoding_block"):
            cur_tensor=tf.layers.conv2d_transpose(cur_tensor,padding='VALID',filters=deconv_filter_count[i],kernel_size= k_s)
            cur_tensor=utils.unpool(cur_tensor)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)

In [ ]:
# serialize graph for tensorboard
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)